In [59]:
import pyspark
from pyspark.sql import SparkSession

import pyspark.sql.functions as f

from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, LongType, FloatType, DoubleType,
    StringType, DateType, TimestampType, BooleanType,
    MapType, ArrayType
)

In [60]:
spark = SparkSession.builder.appName("pyspark-learning").getOrCreate()

In [61]:
df = spark.read.format("csv").options(header=True, delimeter=",").load("./from_slack/customers.csv")
df.createOrReplaceTempView("cust")
df.show(5)

+---+----------+-----------+---------+--------------------+------------+------+
| id|first_name|  last_name|    phone|               email|    acc_type|points|
+---+----------+-----------+---------+--------------------+------------+------+
|  1|    Julian|     Siatka|828223683|juliansiatka@onet.pl|    Ultimate|233101|
|  2|     Tymon|Buszkiewicz|069601377|tymonbuszkiewicz@...|    Ultimate| 61371|
|  3|    Karina|        Roś|160231488| karinaroś@yahoo.com|Professional|215880|
|  4|     Karol|     Marcol|469824688|karolmarcol@inter...|Professional|495433|
|  5| Marcelina|     Gamrat|899504259|marcelinagamrat@o...|Professional|264302|
+---+----------+-----------+---------+--------------------+------------+------+
only showing top 5 rows



In [62]:
# Tworzenie mapy (JSON) - jak API jest wymaga niera i jest body
df2 = df.select(
    f.col("id"),
    f.col("points"),
    f.create_map(
        f.lit("first_name"), f.col("first_name"),
        f.lit("last_name"), f.col("last_name"),
        f.lit("email"), f.col("email"),
        f.lit("acc_type"), f.col("acc_type"),
    ).alias("body")
)

In [63]:
#rozpakowywanie z JSONa body
df2.show(5, truncate=False)
df2.select(
    f.explode(f.col("body"))
).show()

+---+------+-----------------------------------------------------------------------------------------------------------+
|id |points|body                                                                                                       |
+---+------+-----------------------------------------------------------------------------------------------------------+
|1  |233101|{first_name -> Julian, last_name -> Siatka, email -> juliansiatka@onet.pl, acc_type -> Ultimate}           |
|2  |61371 |{first_name -> Tymon, last_name -> Buszkiewicz, email -> tymonbuszkiewicz@interia.pl, acc_type -> Ultimate}|
|3  |215880|{first_name -> Karina, last_name -> Roś, email -> karinaroś@yahoo.com, acc_type -> Professional}           |
|4  |495433|{first_name -> Karol, last_name -> Marcol, email -> karolmarcol@interia.pl, acc_type -> Professional}      |
|5  |264302|{first_name -> Marcelina, last_name -> Gamrat, email -> marcelinagamrat@o2.pl, acc_type -> Professional}   |
+---+------+--------------------

** UDF i mapowanie**

In [64]:
df = spark.read.format("csv").options(header=True, delimeter=",").load("./from_slack/customers.csv")
df.select(f.col("first_name")).show(5)

+----------+
|first_name|
+----------+
|    Julian|
|     Tymon|
|    Karina|
|     Karol|
| Marcelina|
+----------+
only showing top 5 rows



In [65]:
txt = "Aleksandra"
result = ""
for x in range(len(txt)): 
    if x %2 != 0:
            result += txt[x].upper()
    else:
            result += txt[x].lower()  
print(result)
   
    
   

aLeKsAnDrA


In [66]:
def second_upper_char(txt):    
    result = ""
    for x in range(len(txt)):
            if x %2 != 0:
                result += txt[x].upper()
            else:
                result += txt[x].lower()
    return result        

suc_udf = f.udf(second_upper_char)

In [67]:
df.select(suc_udf(f.col("first_name"))).show(5)

+-----------------------------+
|second_upper_char(first_name)|
+-----------------------------+
|                       jUlIaN|
|                        tYmOn|
|                       kArInA|
|                        kArOl|
|                    mArCeLiNa|
+-----------------------------+
only showing top 5 rows



In [68]:
df = spark.read.format("csv").options(header=True, delimeter=",").load("./from_slack/customers.csv")
df.show(5)

+---+----------+-----------+---------+--------------------+------------+------+
| id|first_name|  last_name|    phone|               email|    acc_type|points|
+---+----------+-----------+---------+--------------------+------------+------+
|  1|    Julian|     Siatka|828223683|juliansiatka@onet.pl|    Ultimate|233101|
|  2|     Tymon|Buszkiewicz|069601377|tymonbuszkiewicz@...|    Ultimate| 61371|
|  3|    Karina|        Roś|160231488| karinaroś@yahoo.com|Professional|215880|
|  4|     Karol|     Marcol|469824688|karolmarcol@inter...|Professional|495433|
|  5| Marcelina|     Gamrat|899504259|marcelinagamrat@o...|Professional|264302|
+---+----------+-----------+---------+--------------------+------------+------+
only showing top 5 rows



1. first+last name
2. +48 phone
3. acc_type -> 3 pierwsze znaki
4. points -> zmiana na xxk

**Mapowanie**

In [69]:
def customer_map(r):
    fname = str(r.first_name)
    lname = str(r.last_name)
    lphone = str(r.phone)
    ltype = str(r.acc_type)[:3]
    points = str(r.points)
    
    return pyspark.Row(
        customer=fname + " " + lname,
        phone="+48 " +lphone,
        acc_type=ltype,
        points=str(points // 1000)+"k"
    )

In [70]:
schema= StructType(
    [
        StructField(("customer"), StringType(),False),
        StructField(("phone"), StringType(),False),
        StructField(("acc_type"), StringType(),False),
        StructField("points"), StringType(),False)
    ]
)

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 2 (155537129.py, line 6)

In [ ]:
new_df = df.rdd.map(lambda r: customer_map(r)).toDF(["customer", "phone", "acc_type", "points"])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 19) (e61003ad5388 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pyspark/rdd.py", line 2849, in takeUpToNumLeft
    yield next(iterator)
          ^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pys

In [ ]:
df = spark.read.format("csv").options(header=True, delimeter=",").load("./from_slack/customers.csv")

In [ ]:
def explode_rows(r):
    points = int(r.points)
    return [
        pyspark.Row(id=r.id, wage="K", points=int(points/1000)),
        pyspark.Row(id=r.id, wage="M", points=int(points/1000000)),
        pyspark.Row(id=r.id, wage="B", points=int(points)),
    ]

In [ ]:
schema= StructType(
    [
        StructField("id", StringType(),False),
        StructField("wage", StringType(),False),
        StructField("points", StringType(),False)
    ]
)

In [ ]:
new_df2 = df.rdd.flatMap(lambda r: explode_rows(r))   # lista wierszy zwracana Flat mapa sluzy do rozpakowwania tego 
print(new_df2)

PythonRDD[114] at RDD at PythonRDD.scala:53


In [ ]:
new_df2 = df.rdd.flatMap(lambda r: explode_rows(r)).toDF(schema)   # lista wierszy zwracana Flat mapa sluzy do rozpakowwania tego 
new_df2.show(15)

+---+----+------+
| id|wage|points|
+---+----+------+
|  1|   K|   233|
|  1|   M|     0|
|  1|   B|233101|
|  2|   K|    61|
|  2|   M|     0|
|  2|   B| 61371|
|  3|   K|   215|
|  3|   M|     0|
|  3|   B|215880|
|  4|   K|   495|
|  4|   M|     0|
|  4|   B|495433|
|  5|   K|   264|
|  5|   M|     0|
|  5|   B|264302|
+---+----+------+
only showing top 15 rows



Zapisywanie danych

na dysku


In [ ]:
df = spark.read.format("csv").options(header=True, delimeter=",").load("./from_slack/customers.csv")
df.write.mode("overwrite").format("csv").save("./from_slack/df2_order_details")

In [ ]:
new_df2.write.mode("overwrite").format("csv").save("./from_slack/new_df2")

partition + lub -  // definiuje ilosc plikow ale i sposob partycjonowania np po dacie 

coalasce tylko - // definiuje ilosc plikow

In [73]:
df.repartition("acc_type").write.mode("overwrite").format("csv").save("./from_slack/order_by_type")

CACHE / PERSIST


In [74]:
df = spark.read.format("csv").options(header=True, delimeter=",").load("./from_slack/order_details.csv")
df = df.where(f.col("quantity") <= 3)
df = df.where(f.col("order_id") <= 45000)
df = df.orderBy(f.col("product_id").desc())
df.show(5)

+--------+----------+--------+
|order_id|product_id|quantity|
+--------+----------+--------+
|   36841|      9999|       2|
|   36983|      9999|       2|
|   33413|      9998|       3|
|   27932|      9996|       3|
|   36705|      9996|       1|
+--------+----------+--------+
only showing top 5 rows



In [75]:
#zapis do
df.cache()
df.persist()

DataFrame[order_id: string, product_id: string, quantity: string]

In [76]:
from pyspark import StorageLevel

In [ ]:
df.persist(StorageLevel.MEMORY_ONLY)   # to samo co cache
df.persist(StorageLevel.MEMORY_AND_DISK)   # jak sie skonczy w memory only to na dysku zapisuje ale warto miec duzy dysk
df.persist(StorageLevel.MEMORY_ONLY_SER)        # mniej miejsca niz powyzsze ale bardziej obciaza CPU bo odczyt bedzie wymagac wiecej pracy
df.persist(StorageLevel.MEMORY_AND_DISK_SER)    # to samo ale tez dysk jak memory za male
df.persist(StorageLevel.DISK_ONLY)              #nie mylic z write to zapis w pamieci

#replikacja dowolnej opci 
df.persist(StorageLevel.MEMORY_ONLY_2)   

In [ ]:
#wyczyszczenie tylko do persist bo 
#   casch przechowuje w pamieci ram do memory only
#   persist ma dodatkowe mozliwosci 
df.unpersist()